<a href="https://colab.research.google.com/github/yacoan81/motorEvoked/blob/main/02_Modelling_GB%2BADA%2BSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies and Packages

https://scikit-learn.org/stable/modules/calibration.html
https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html#sphx-glr-auto-examples-calibration-plot-calibration-curve-py

https://www.kaggle.com/residentmario/notes-on-classification-probability-calibration
https://www.kaggle.com/cbourguignat/why-calibration-works
https://www.kaggle.com/dowakin/probability-calibration-0-005-to-lb
https://www.kaggle.com/vbmokin/cfec-probability-calibration
https://www.kaggle.com/kkhandekar/what-is-probability-calibration-sklearn

In [ ]:
# Run once, to be able to use oversampling
# pip install -U imbalanced-learn
!pip install ml_insights


In [ ]:
# sampling with imbalance data
# https://github.com/scikit-learn-contrib/imbalanced-learn#id32
from imblearn.over_sampling import RandomOverSampler
from imblearn import FunctionSampler  # to use a idendity sampler
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import scipy as sp
import math
import warnings
import matplotlib.pyplot as plt 
import ml_insights as mli

warnings.filterwarnings('ignore')

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import (train_test_split, GroupShuffleSplit, 
                                     GroupKFold, GridSearchCV, cross_val_predict,
                                     LeaveOneGroupOut, RandomizedSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score, balanced_accuracy_score, roc_auc_score,
                             log_loss, make_scorer, accuracy_score, roc_curve,
                             auc)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.preprocessing import StandardScaler

from imblearn.ensemble import BalancedRandomForestClassifier



/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Functions

def bins(prob,nbin):
  val= []
  for low in np.linspace(0,1,nbin):
    valz = prob[:,1][(prob[:,1]>= low) & (prob[:,1]<= low+(1/nbin))]
    val.append(len(valz))
  # print("Boundaries: {} - {} | freq: {}".format(low,low+0.1,len(val)))
  return val;

def reliability(name,y,probabilities,bins,strategy='uniform'):
  fraction_of_positives, mean_predicted_value = calibration_curve(y, probabilities[:,1], n_bins=bins, strategy=strategy)
  # bins_cal = len(fraction_of_positives)
  # bin_counts = pd.cut(probabilities[:,1], bins=bins_cal).value_counts()
  # ece = np.sum(np.abs(fraction_of_positives - mean_predicted_value) * (bin_counts / len(y_test)))
  ece = expected_calibration_error(y=y, proba=probabilities[:,1], bins=bins)

  fig = plt.figure(figsize=(10, 10))
  ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
  ax2 = plt.subplot2grid((3, 1), (2, 0))
  
  ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s | ECE: (%1.3f)" % (name, ece))

  ax2.hist(probabilities[:,1], range=(0, 1), bins=bins, label=name,histtype="step", lw=2)
  # ax2.hist(probabilities[:,1], range=(probabilities[:,1].min(), probabilities[:,1].max()), bins=bins, label=name,histtype="step", lw=2)
  ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
  
  ax1.set_ylabel("Fraction of positives")
  # ax1.set_xlim([probabilities[:,1].min(), probabilities[:,1].max()])
  ax1.set_ylim([-0.05, 1.05])
  ax1.legend(loc="lower right")
  ax1.set_title('Calibration plots  (reliability curve)')

  ax2.set_xlabel("Mean predicted value")
  ax2.set_ylabel("Count")
  ax2.legend(loc="upper center", ncol=2)

  plt.tight_layout()


def expected_calibration_error(y, proba, bins):
  bin_count, bin_edges = np.histogram(proba, bins = bins)
  n_bins = len(bin_count)
  bin_edges[0] -= 1e-8 # because left edge is not included
  bin_id = np.digitize(proba, bin_edges, right = True) - 1
  bin_ysum = np.bincount(bin_id, weights = y, minlength = n_bins)
  bin_probasum = np.bincount(bin_id, weights = proba, minlength = n_bins)
  bin_ymean = np.divide(bin_ysum, bin_count, out = np.zeros(n_bins), where = bin_count > 0)
  bin_probamean = np.divide(bin_probasum, bin_count, out = np.zeros(n_bins), where = bin_count > 0)
  ece = np.abs((bin_probamean - bin_ymean) * bin_count).sum() / len(proba)
  return ece

def get_group_split(groupvector, test_size):
    """split items according to their group label
    Input:
         groupvector  -- labels of the item groups
         test_size        -- desired size of the test set
    Output:
        train_keys      -- group labels of the items in the train set
        test_keys       -- group labels of the items in the test set
    """
    if type(test_size) is float:
        test_size = round(test_size*len(groupvector))

    # count number of items in each group
    gr_counts = Counter(groupvector)
    # randomly shuffle group keys
    gr_keys = np.random.choice(np.asarray(list(gr_counts.keys())), 
                                                    size = len(gr_counts), replace=False)

    # find the index of the group in the list `gr_keys` that optimally splits the set 
    # according to the specified `test_size`
    # to this end: 
    #   (1) calculate cumulative sum of items in the ordered list of groups
    cum_num_items = np.cumsum(np.array([gr_counts[x] for x in gr_keys]))
    # (2) find the group index so that the splitting yields the set of the size that is closest to the specified
    ind = np.argmin(abs(cum_num_items - test_size))
    if ind > (len(gr_counts) -1):
        print("split index", ind)
        print(
            abs(np.cumsum(np.array([gr_counts[x] for x in gr_keys])) - test_size)
            )
        raise ValueError("Unable to split")

    # split groups according to the obtained index
    test_keys = [kk for nn, kk in enumerate(gr_keys) if nn<=ind]
    train_keys = [kk for nn, kk in enumerate(gr_keys) if nn>ind]
    return train_keys, test_keys

def iterate_models(model,n,testSize):
  n = n
  brier = np.zeros(n)
  logloss = np.zeros(n)
  ece =  np.zeros(n)
  auc =  np.zeros(n)

  brier_iso = np.zeros(n)
  logloss_iso = np.zeros(n)
  ece_iso =  np.zeros(n)
  auc_iso =  np.zeros(n)

  brier_sig = np.zeros(n)
  logloss_sig = np.zeros(n)
  ece_sig =  np.zeros(n)
  auc_sig =  np.zeros(n)

  proportion_train = np.zeros(n)
  proportion_test = np.zeros(n)
  for i in range(n):
    train_keys,test_keys =  get_group_split(groups_all, test_size = testSize)

    test_mask = [x in set(test_keys) for x in groups_all]
    X_test_all =  X[test_mask]
    X_test_ = X_test_all.iloc[:,1:]
    xxtest = X_test_all.patient_uid.nunique()
    y_test_ =  y[test_mask]

    train_mask = [x in set(train_keys) for x in groups_all]
    X_train_all =  X[train_mask]
    X_train_ = X_train_all.iloc[:,1:]
    xxtrain = X_train_all.patient_uid.nunique()
    y_train_ =  y[train_mask]
    
    clf = model.fit(X_train_, y_train_)

    ########################Isotonic Calibration################################
    clf_iso = CalibratedClassifierCV(base_estimator=model, cv=4, method='isotonic')
    clf_iso.fit(X_train_, y_train_)
    proba_iso = clf_iso.predict_proba(X_test_)

    auc_iso[i] = roc_auc_score(y_test_, proba_iso[:,1])
    logloss_iso[i] = log_loss(y_test_, proba_iso)
    brier_iso[i] = brier_score_loss(y_test_, proba_iso[:,1])
    ece_iso[i] = expected_calibration_error(y_test_, proba_iso[:,1],10)
    ########################sigmoid Calibration################################
    clf_sig = CalibratedClassifierCV(base_estimator=model, cv=4, method='sigmoid')
    clf_sig.fit(X_train_, y_train_)
    proba_sig = clf_sig.predict_proba(X_test_)

    auc_sig[i] = roc_auc_score(y_test_, proba_sig[:,1])
    logloss_sig[i] = log_loss(y_test_, proba_sig)
    brier_sig[i] = brier_score_loss(y_test_, proba_sig[:,1])
    ece_sig[i] = expected_calibration_error(y_test_, proba_sig[:,1],10)
    ############################################################################
    y_pred = clf.predict(X_test_)
    proba = clf.predict_proba(X_test_)

    brier[i] = brier_score_loss(y_test_, proba[:,1])
    logloss[i] = log_loss(y_test_, proba)
    ece[i] = expected_calibration_error(y_test_, proba[:,1],10)
    auc[i] = roc_auc_score(y_test_, proba[:,1])
    
    proportion_train[i] = xxtest/groups_all.nunique()
    proportion_test[i]  = xxtrain/groups_all.nunique()
  return auc, logloss, brier, ece, proportion_train, proportion_test,\
         auc_iso,logloss_iso,brier_iso,ece_iso,\
         auc_sig,logloss_sig,brier_sig,ece_sig


# Import Data

In [ ]:

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

final = '18ItiyhgcSV61_OHKV8tMCiXDnGXPYPBa'

In [ ]:
downloaded = drive.CreateFile({'id':final}) 
downloaded.GetContentFile('final.csv')  
final = pd.read_csv('final.csv')

In [ ]:
final_Copy = final.copy()

In [ ]:
final.shape

In [ ]:
# final.head()

In [ ]:
#removing imported column
final.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
Counter(final['w'])

In [ ]:
print("proportion of positive outcomes: {}".format(final[(final["w"]==1)].size / final.size * 100))

In [ ]:
final.corr()

# Preparation

In [ ]:
final.columns

In [ ]:
final_=pd.get_dummies(final, columns=["sex"])
final_.drop('sex_Female', axis=1, inplace=True)

In [ ]:
final_['visit_date'] = pd.to_datetime(final_['visit_date'], format='%Y/%m/%d', errors='coerce')
final_['dateT0'] = pd.to_datetime(final_['dateT0'], format='%Y/%m/%d', errors='coerce')
final_['dateT1'] = pd.to_datetime(final_['dateT1'], format='%Y/%m/%d', errors='coerce')
final_['date_of_birth'] = pd.to_datetime(final_['date_of_birth'], format='%Y/%m/%d', errors='coerce')

In [ ]:
final_.dtypes

In [ ]:
# how much visits disappear if you filter on "MEP visit outside 1 year of EDSS visit" 
Counter((final_['dateT0'] - final_['visit_date']).dt.days.abs() > 365)

In [ ]:
cols = ['patient_uid','age','edss0','sex_Male','AH-L','AH-R','APB-L','APB-R','diff_AH-L','diff_AH-R','diff_APB-L','diff_APB-R']

In [ ]:
X = final_[cols]
y = final_['w'].astype('int')

In [ ]:
X.head()

## Split Train and Test Sets

We need to split between train and test sosame  patient is not in both splits \\


In [ ]:
groups_all = X['patient_uid']
groups_all.nunique()

In [ ]:
## 
train_idx, test_idx = next(GroupShuffleSplit(train_size=.8, random_state=42).split(X, y, groups_all))

In [ ]:
X_train = np.take(X, train_idx, axis=0)
y_train = np.take(y, train_idx, axis=0)
groups_X_train = X_train['patient_uid']

X_test = np.take(X, test_idx, axis=0)
y_test = np.take(y, test_idx, axis=0)

In [ ]:
# taking the patients (groups) apart in case is needed later on
groups_X_test = X_test['patient_uid']
X_test = X_test.iloc[:,1:]

In [ ]:
X_train.shape, groups_X_train.nunique()

In [ ]:
X_test.shape, groups_X_test.nunique()

## Scaling

In [ ]:
# Remark: Scaling in training set and apply fit in both train and test


In [ ]:
# cols to scale, although we can apply it in all
col_names = ['age', 'edss0','AH-L',	'AH-R',	'APB-L',	'APB-R', 'diff_AH-L', 'diff_AH-R', 'diff_APB-L', 'diff_APB-R']
features = X_train[col_names]
# scaler contains the scaling function 
scaler = StandardScaler().fit(features.values)

X_train_scaled = scaler.transform(X_train[col_names].values)
X_test_scaled  = scaler.transform(X_test[col_names].values)

X_train[col_names] = X_train_scaled
X_test[col_names] = X_test_scaled


In [ ]:
Counter(y_train), Counter(y_test)

## Oversampling

In [ ]:
def func(X, y, sampling_strategy, random_state):
  return RandomOverSampler(
      sampling_strategy=sampling_strategy,
      random_state=random_state).fit_resample(X, y)

sampler = FunctionSampler(func=func,
                          kw_args={'sampling_strategy': 'auto',
                                   'random_state': 42})
X_train_overSampled, y_train_overSampled = sampler.fit_resample(X_train, y_train)

In [ ]:
# RandomOverSampler returns numpy ndarray, transform to dataframe 
X_train_overSampled = pd.DataFrame(X_train_overSampled)
X_train_overSampled.columns =  cols
X_train_overSampled['sex_Male'] = X_train_overSampled['sex_Male'].astype(int)
X_train_overSampled['patient_uid'] = X_train_overSampled['patient_uid'].astype(int)

groups_X_train_overSampled = X_train_overSampled['patient_uid']

In [ ]:
# In case we want to try, split the train set into train and calibration, for both oversampled and non-oversampled

train_idx_1, test_idx_1 = next(GroupShuffleSplit(train_size=.50, random_state=42).split(X_train, y_train, groups_X_train))
X_train_cali = np.take(X_train, train_idx_1, axis=0)
y_train_cali = np.take(y_train, train_idx_1, axis=0)
X_calib = np.take(X_train, test_idx_1, axis=0)
y_calib = np.take(y_train, test_idx_1, axis=0)
groups_X_train_cali = X_train_cali['patient_uid']
groups_X_calib = X_calib['patient_uid']
X_train_cali = X_train_cali.iloc[:,1:]
X_calib = X_calib.iloc[:,1:]

# with oversampled train data
train_idx_2, test_idx_2 = next(GroupShuffleSplit(train_size=.50, random_state=42).split(X_train_overSampled, y_train_overSampled, groups_X_train_overSampled))
X_train_2 = np.take(X_train_overSampled, train_idx_2, axis=0)
y_train_2 = np.take(y_train_overSampled, train_idx_2, axis=0)
X_calib_2 = np.take(X_train_overSampled, test_idx_2, axis=0)
y_calib_2 = np.take(y_train_overSampled, test_idx_2, axis=0)
groups_X_train_2 = X_train_2['patient_uid']
groups_X_calib_2 = X_calib_2['patient_uid']
X_train_2 = X_train_2.iloc[:,1:]
X_calib_2 = X_calib_2.iloc[:,1:]

groups_X_train = X_train['patient_uid']
X_train = X_train.iloc[:,1:]

groups_X_train_overSampled = X_train_overSampled['patient_uid']
X_train_overSampled = X_train_overSampled.iloc[:,1:]



# dummy classifier

In [ ]:
clf_dummy = DummyClassifier(strategy='most_frequent', random_state=42)
clf_dummy.fit(X_train, y_train)

In [ ]:
y_pred_dummy = clf_dummy.predict(X_test)
proba_dummy = clf_dummy.predict_proba(X_test)

In [ ]:
ece = expected_calibration_error(y_test,proba_dummy[:,1],20)


In [ ]:
print("\tscore: %1.3f" % clf_dummy.score(X_test, y_test))
print("\tF1: %1.3f (best is 1, worst is 0)" % f1_score(y_test, y_pred_dummy))
print("\tPrecision: %1.3f (best is 1, worst is 0)" % precision_score(y_test, y_pred_dummy))
print("\tRecall: %1.3f (best is 1, worst is 0)" % recall_score(y_test, y_pred_dummy))
print("\tbal_acc: %1.3f (best is 1, worst is 0)" % balanced_accuracy_score(y_test, y_pred_dummy))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_dummy[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_dummy))
print("\tBrier: %1.3f (smaller the better)" % brier_score_loss(y_test, proba_dummy[:,1]))
print("\tECE: %1.3f (smaller the better)" % expected_calibration_error(y_test, proba_dummy[:,1],10))

In [ ]:
reliability("Dummy classifier", y_test, proba_dummy,10)

# Gradient Boosting

## Tunning Gradient Boosting

In [ ]:
# learning rate shrinks the contribution of each tree by learning_rate.
learning_rate = [1, 0.5, 0.25, 0.1, 0.05, 0.01]
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
# max_depth = np.linspace(1, 32, 32, endpoint=True)
max_depth = [int(x) for x in np.linspace(1, 32, 32)]
max_depth.append(None)
# Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
min_samples_split = np.linspace(0.1, 1.0, 10, endpoint=True)
# Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
min_samples_leaf = np.linspace(0.1, 0.5, 5, endpoint=True)
# Create the random grid
random_grid = {'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

cv = GroupKFold(n_splits=5)

In [ ]:
# #############################################
# ##### Execution takes around 18 minutes #####
# #############################################

# Model_GB = GradientBoostingClassifier(random_state=42)
# clf_GB = RandomizedSearchCV(estimator = Model_GB, param_distributions = random_grid, n_iter = 100, cv = cv, scoring = 'brier_score_loss', verbose=2, random_state=42, n_jobs = -1)
# # # Fit the random search model
# clf_GB.fit(X_train, y_train, groups_X_train)

# print(clf_GB.best_score_)
# print(clf_GB.best_params_)

In [ ]:
best_params_GB = {'n_estimators': 235, 'min_samples_split': 0.8, 'min_samples_leaf': 0.4, 'max_features': 'sqrt', 'max_depth': 25, 'learning_rate': 0.01}
# best_params_GB = clf_GB.best_params_

In [ ]:
# Fit model with best parameters
Model_GB = GradientBoostingClassifier(random_state=42)
Model_GB.set_params(**best_params_GB)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=25,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=0.4, min_samples_split=0.8,
                           min_weight_fraction_leaf=0.0, n_estimators=235,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
auc_gb,logloss_gb,brier_gb,ece_gb,prop_train_gb,prop_test,auc_iso,logloss_iso_gb,brier_iso_gb,ece_iso_gb,auc_sig_gb,logloss_sig_gb,brier_sig_gb,ece_sig_gb = iterate_models(Model_GB,100,0.2)

In [ ]:
# np.mean(auc_gb)
auc_gb

array([0.64505936, 0.66703495, 0.63063789, 0.60037813, 0.71688144,
       0.65432373, 0.56767936, 0.72876201, 0.60556888, 0.66199428,
       0.66843726, 0.67836949, 0.6508658 , 0.63160173, 0.65249145,
       0.614421  , 0.68810764, 0.63033234, 0.7111749 , 0.6222363 ,
       0.54320847, 0.57563978, 0.7175853 , 0.72567135, 0.62075703,
       0.67038074, 0.60674541, 0.70762809, 0.64768923, 0.63418079,
       0.64412133, 0.58265583, 0.66088932, 0.49788071, 0.570199  ,
       0.5679446 , 0.60867782, 0.61298715, 0.60180234, 0.62162439,
       0.71061288, 0.59836951, 0.65637423, 0.60958005, 0.60651209,
       0.66351719, 0.65942391, 0.70611455, 0.6860886 , 0.60909011,
       0.63684371, 0.59020147, 0.68418676, 0.63856646, 0.68229167,
       0.65126931, 0.62673288, 0.64606809, 0.64247567, 0.60997293,
       0.57560494, 0.59981083, 0.65108429, 0.67862853, 0.71961806,
       0.66637827, 0.63557273, 0.52678877, 0.61945721, 0.62406799,
       0.62186028, 0.6443081 , 0.60845209, 0.62241285, 0.58898

In [ ]:
auc_iso

array([0.66150421, 0.67061738, 0.63186211, 0.57801047, 0.7162049 ,
       0.63177067, 0.58020729, 0.74303587, 0.60131226, 0.6512478 ,
       0.69188459, 0.67935569, 0.64641311, 0.62065553, 0.66478691,
       0.60622294, 0.69137731, 0.64409722, 0.71102383, 0.61995565,
       0.53509974, 0.58916424, 0.72312628, 0.7222715 , 0.63250329,
       0.65881859, 0.62296588, 0.71491077, 0.65133355, 0.63871648,
       0.64551559, 0.5938634 , 0.65364908, 0.48657065, 0.57502488,
       0.56247491, 0.61554024, 0.61543563, 0.59989602, 0.63947422,
       0.70844875, 0.62420365, 0.65540353, 0.62122703, 0.60859524,
       0.65713313, 0.66670829, 0.70743034, 0.67684318, 0.62140603,
       0.64017094, 0.58964184, 0.66365594, 0.6438455 , 0.65734954,
       0.65771853, 0.62259087, 0.64232199, 0.64039877, 0.6132029 ,
       0.59469136, 0.61143469, 0.65256751, 0.69710814, 0.72841435,
       0.66709925, 0.62887031, 0.52213854, 0.62768458, 0.61282067,
       0.60102695, 0.64348319, 0.60257126, 0.63978369, 0.59194

In [ ]:
auc_sig_gb

array([0.66224627, 0.66984733, 0.63566366, 0.58833624, 0.72193943,
       0.64130504, 0.58569866, 0.73470657, 0.60262923, 0.65317172,
       0.68465552, 0.66472715, 0.64140383, 0.61648114, 0.6631922 ,
       0.61501623, 0.6817419 , 0.64053199, 0.713851  , 0.61362053,
       0.5405817 , 0.57720549, 0.72787985, 0.70527223, 0.62421048,
       0.65629899, 0.61805774, 0.70584341, 0.65603067, 0.62717832,
       0.64580014, 0.5866979 , 0.66616913, 0.49128498, 0.55681592,
       0.57108089, 0.59891732, 0.63101918, 0.59566251, 0.62131751,
       0.7042287 , 0.62236799, 0.65489507, 0.61206802, 0.60039851,
       0.65617552, 0.66670829, 0.69692982, 0.68093477, 0.61364832,
       0.63470696, 0.60286935, 0.67206281, 0.6312873 , 0.66487269,
       0.66088153, 0.61906171, 0.64843563, 0.64419653, 0.59846807,
       0.57708642, 0.6132268 , 0.6633081 , 0.6924711 , 0.71964699,
       0.66130738, 0.63704463, 0.53198106, 0.6255476 , 0.61888664,
       0.60658687, 0.64840689, 0.60199176, 0.63517351, 0.58686

In [ ]:
xxxx = np.array([0.62363466, 0.60864734, 0.58155895, 0.6510472 , 0.5646533 ,
       0.6114505 , 0.63602785, 0.63726656, 0.597318  , 0.63595879,
       0.59561671, 0.62935596, 0.64173954, 0.60397648, 0.63289683,
       0.62240391, 0.6422959 , 0.61363135, 0.6352066 , 0.60632258,
       0.6315532 , 0.529654  , 0.60188249, 0.64770036, 0.62068486,
       0.60823389, 0.60862565, 0.56947386, 0.64827423, 0.61470714,
       0.5397693 , 0.62648557, 0.63939732, 0.63277758, 0.63960158,
       0.6071587 , 0.62430453, 0.62658485, 0.60719778, 0.62480615,
       0.63198078, 0.60858107, 0.58417089, 0.61764233, 0.63516894,
       0.63039404, 0.62459419, 0.60900401, 0.61866166, 0.6261739 ,
       0.62684566, 0.58598462, 0.61007119, 0.65670784, 0.52148824,
       0.61332623, 0.63634951, 0.63071563, 0.63461311, 0.6228628 ,
       0.5794488 , 0.63173152, 0.64299791, 0.63575601, 0.64702958,
       0.66283064, 0.63055825, 0.59898191, 0.59735801, 0.64092   ,
       0.61480658, 0.62438889, 0.62103226, 0.63765133, 0.62680826,
       0.61006226, 0.6348314 , 0.62454755, 0.63906094, 0.64079077,
       0.62658223, 0.62064185, 0.62729029, 0.6325209 , 0.62243156,
       0.63694798, 0.62407831, 0.63631933, 0.65062499, 0.62508973,
       0.63010916, 0.62576209, 0.61406711, 0.63168632, 0.61812927,
       0.6487796 , 0.42937596, 0.620663  , 0.63331484, 0.60854854])

In [ ]:
xxxx

In [ ]:
for size in [0.2,0.3,0.5,0.8]:
  auc,logloss,brier,ece,prop_train,prop_test,auc_iso,logloss_iso,brier_iso,ece_iso,auc_sig,logloss_sig,brier_sig,ece_sig = iterate_models(Model_GB,1000,size)
  print('*'*70)

  print("\tTest Set Size: %.2f\n\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (size,
                                                                                                                           auc.mean(),auc.std(),
                                                                                                                           logloss.mean(),logloss.std(),
                                                                                                                           brier.mean(),brier.std(),
                                                                                                                           ece.mean(),ece.std()) )
  print("\n")
  print("\tAUC(min): %1.3f      \tAUC(max): %1.3f"    % (auc.min(),auc.max()))
  print("\tlog_loss(min): %1.3f  \tlog_loss(max): %1.3f" % (logloss.min(),logloss.max()))
  print("\tBrier(min): %1.3f    \tBrier(max): %1.3f"   % (brier.min(),brier.max()))
  print("\tECE(min): %1.3f       \tECE(max): %1.3f"      % (ece.min(),ece.max()))
  print("\tProp train: %.2f \tProp test: %.2f" % (prop_train.mean(), prop_test.mean()))  
  print("\nIsotonic Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_iso.mean(),auc_iso.std(),
                                                                                                                           logloss_iso.mean(),logloss_iso.std(),
                                                                                                                           brier_iso.mean(),brier_iso.std(),
                                                                                                                           ece_iso.mean(),ece_iso.std()) )
  print("\nSigmoid Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_sig.mean(),auc_sig.std(),
                                                                                                                           logloss_sig.mean(),logloss_sig.std(),
                                                                                                                           brier_sig.mean(),brier_sig.std(),
                                                                                                                           ece_sig.mean(),ece_sig.std()) )

In [ ]:
clf_GB = Model_GB.fit(X_train, y_train)

y_pred_GB = clf_GB.predict(X_test)
proba_GB_uncalibrated = clf_GB.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_GB.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_GB))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_GB))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_GB))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_GB))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_GB_uncalibrated[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_GB_uncalibrated))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_GB_uncalibrated[:,1]))

## Reliability plots

In [ ]:
reliability("Grandient Boosting",y_test, proba_GB_uncalibrated, 10, 'quantile')

In [ ]:
# first column is the average predicted probability, second is the empirical probability, and last the number of observations
# pd.concat([pd.DataFrame(rd['pred_probs']),pd.DataFrame(rd['emp_probs']),pd.DataFrame(rd['bin_counts'])], axis=1, join="inner")

## Isotonic


In [ ]:
# Fit model with best parameters
Model_GB_cali = GradientBoostingClassifier(random_state=42)
Model_GB_cali.set_params(**best_params_GB)

In [ ]:
# Calibrated with Isotonic Calibration 

# Using CV from calibration - with oversampled train data
clf_GB_iso = CalibratedClassifierCV(base_estimator=Model_GB_cali, cv=4, method='isotonic')
clf_GB_iso.fit(X_train, y_train)

###############################################
y_pred_GB_iso = clf_GB_iso.predict(X_test)
proba_GB_iso = clf_GB_iso.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_GB_iso.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_GB_iso))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_GB_iso))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_GB_iso))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_GB_iso))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_GB_iso[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_GB_iso))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_GB_iso[:,1]))

In [ ]:
reliability("Gradient Boosting",y_test, proba_GB_iso, 10, 'quantile')

In [ ]:
# first column is the average predicted probability, second is the empirical, and last the number of observations
# pd.concat([pd.DataFrame(rd['pred_probs']),pd.DataFrame(rd['emp_probs']),pd.DataFrame(rd['bin_counts'])], axis=1, join="inner")

## Platt Scaling


In [ ]:
# Calibrated with Sigmnoid Calibration 

# Using CV from calibration - with train data
clf_GB_sigmoid = CalibratedClassifierCV(base_estimator=Model_GB_cali, cv=4, method='sigmoid')
clf_GB_sigmoid.fit(X_train, y_train)

###############################################
y_pred_GB_sigmoid = clf_GB_sigmoid.predict(X_test)
proba_GB_sigmoid = clf_GB_sigmoid.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_GB_sigmoid.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_GB_sigmoid))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_GB_sigmoid))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_GB_sigmoid))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_GB_sigmoid))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_GB_sigmoid[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_GB_sigmoid))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_GB_sigmoid[:,1]))

In [ ]:
reliability("GB Calibrated (training)", y_train, clf_GB_sigmoid.predict_proba(X_train), 10, 'quantile')

In [ ]:
reliability("GB uncalibrated", y_test, proba_GB_uncalibrated, 10, 'quantile')

In [ ]:
reliability("GB Calibrated Isotonic", y_test, proba_GB_iso, 10,'quantile')

In [ ]:
reliability("GB Calibrated Sigmoid", y_test, proba_GB_sigmoid, 10, 'quantile')

# AdaBoost

## GridSearch CV

In [ ]:
# p_grid = {  
#     'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 2000, num = 15)],
#     'learning_rate':[1, 0.5, 0.25, 0.1, 0.05, 0.01]
#     }

# cv = GroupKFold(n_splits=4)

# Model_Ada = AdaBoostClassifier(random_state=42)

# clf_Ada = RandomizedSearchCV(estimator = Model_Ada, param_distributions = p_grid, n_iter = 100, cv = cv, scoring = 'brier_score_loss', verbose=2, random_state=42, n_jobs = -1)
# clf_Ada.fit(X_train, y_train, groups_X_train)

# print(clf_Ada.best_score_)
# print(clf_Ada.best_params_)

In [ ]:
best_para_ada = {'n_estimators': 100, 'learning_rate': 0.01}
# best_para_ada = clf_Ada.best_params_

In [ ]:
# Fit model with best parameters
Model_Ada_refit = AdaBoostClassifier(random_state=42)
Model_Ada_refit.set_params(**best_para_ada)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.01,
                   n_estimators=100, random_state=42)

In [ ]:
auc_ada,logloss_ada,brier_ada,ece_ada,prop_train,prop_test,auc_iso_ada,logloss_iso_ada,brier_iso_ada,ece_iso_ada,auc_sig_ada,logloss_sig_ada,brier_sig_ada,ece_sig_ada = iterate_models(Model_Ada_refit,100,0.2)

In [ ]:
# np.mean(auc_ada)
auc_ada

array([0.6316361 , 0.66607763, 0.74116821, 0.64682895, 0.6153808 ,
       0.63583112, 0.60101756, 0.60981781, 0.6275652 , 0.62007266,
       0.56290541, 0.7555641 , 0.64953456, 0.71506215, 0.60082849,
       0.67329545, 0.71048268, 0.66572398, 0.62278337, 0.66294171,
       0.55151908, 0.68791667, 0.75826972, 0.59352774, 0.60853678,
       0.63321789, 0.63313802, 0.59767662, 0.65168773, 0.59811098,
       0.70542124, 0.62326416, 0.60899394, 0.61898396, 0.62442283,
       0.60643179, 0.56829151, 0.64700346, 0.56448718, 0.62587246,
       0.52921671, 0.63829787, 0.64794847, 0.6240432 , 0.70163951,
       0.59935153, 0.60467073, 0.58067292, 0.64086673, 0.66825157,
       0.71904509, 0.57171047, 0.62036943, 0.70149597, 0.62015993,
       0.69298084, 0.71537627, 0.59833993, 0.65371448, 0.5447103 ,
       0.67363865, 0.57149123, 0.60888278, 0.57995975, 0.57087071,
       0.61430446, 0.68688218, 0.61337964, 0.64926522, 0.46517918,
       0.64297245, 0.6355    , 0.67407016, 0.6633658 , 0.56055

In [ ]:
auc_iso_ada

array([0.65415058, 0.70216394, 0.72779506, 0.64986559, 0.6556201 ,
       0.6331786 , 0.5915004 , 0.60634278, 0.63989764, 0.63633061,
       0.58740991, 0.72066667, 0.66198752, 0.73084725, 0.63268993,
       0.69211648, 0.71161109, 0.67027458, 0.62739396, 0.68689218,
       0.55667098, 0.70019231, 0.75330378, 0.59803224, 0.63514318,
       0.64239538, 0.6266276 , 0.6185633 , 0.65595257, 0.61062574,
       0.70998098, 0.63096167, 0.61898182, 0.61868687, 0.63137342,
       0.62380973, 0.58778958, 0.65222539, 0.56378205, 0.6247224 ,
       0.52958225, 0.65920763, 0.67014424, 0.63597043, 0.7195298 ,
       0.61181281, 0.63440673, 0.57981249, 0.65605778, 0.69793711,
       0.69596817, 0.57444271, 0.61619433, 0.70659762, 0.62352694,
       0.70419584, 0.70712237, 0.59501429, 0.66389935, 0.5301976 ,
       0.68332012, 0.58722222, 0.63247863, 0.62642045, 0.58073879,
       0.61183727, 0.7013759 , 0.62068861, 0.6694221 , 0.48751755,
       0.64844617, 0.60089474, 0.64747488, 0.67104978, 0.56094

In [ ]:
auc_sig_ada

array([0.65120656, 0.70017981, 0.74616118, 0.65447033, 0.6361831 ,
       0.64030725, 0.61438547, 0.58383941, 0.6463807 , 0.6373297 ,
       0.58193694, 0.74530769, 0.65569915, 0.73636033, 0.61794906,
       0.69155421, 0.73008014, 0.68269231, 0.6414385 , 0.68495923,
       0.5549635 , 0.69913462, 0.74714766, 0.63100996, 0.64569913,
       0.64683983, 0.64485677, 0.62102892, 0.68146357, 0.60847107,
       0.70720633, 0.62269205, 0.60700606, 0.61388394, 0.60924452,
       0.61894044, 0.56925676, 0.64172081, 0.57291667, 0.65367227,
       0.52749347, 0.66307354, 0.64591794, 0.61460627, 0.73437822,
       0.61091754, 0.63669858, 0.58082127, 0.67665424, 0.6841761 ,
       0.70676393, 0.55896001, 0.60323887, 0.69884925, 0.62450898,
       0.67863394, 0.70123227, 0.63297054, 0.65123245, 0.53855171,
       0.68636003, 0.55491228, 0.63910256, 0.61467211, 0.58287599,
       0.60351706, 0.71106065, 0.61951072, 0.65705987, 0.48798565,
       0.64473817, 0.61255263, 0.65730213, 0.67245671, 0.56036

In [ ]:
for size in [0.2,0.3,0.5,0.8]:
  auc,logloss,brier,ece,prop_train,prop_test,auc_iso,logloss_iso,brier_iso,ece_iso,auc_sig,logloss_sig,brier_sig,ece_sig = iterate_models(Model_Ada_refit,1000,size)
  print('*'*70)

  print("\tTest Set Size: %.2f\n\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (size,
                                                                                                                           auc.mean(),auc.std(),
                                                                                                                           logloss.mean(),logloss.std(),
                                                                                                                           brier.mean(),brier.std(),
                                                                                                                           ece.mean(),ece.std()) )
  print("\n")
  print("\tAUC(min): %1.3f      \tAUC(max): %1.3f"    % (auc.min(),auc.max()))
  print("\tlog_loss(min): %1.3f  \tlog_loss(max): %1.3f" % (logloss.min(),logloss.max()))
  print("\tBrier(min): %1.3f    \tBrier(max): %1.3f"   % (brier.min(),brier.max()))
  print("\tECE(min): %1.3f       \tECE(max): %1.3f"      % (ece.min(),ece.max()))
  print("\tProp train: %.2f \tProp test: %.2f" % (prop_train.mean(), prop_test.mean()))  
  print("\nIsotonic Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_iso.mean(),auc_iso.std(),
                                                                                                                           logloss_iso.mean(),logloss_iso.std(),
                                                                                                                           brier_iso.mean(),brier_iso.std(),
                                                                                                                           ece_iso.mean(),ece_iso.std()) )
  print("\nSigmoid Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_sig.mean(),auc_sig.std(),
                                                                                                                           logloss_sig.mean(),logloss_sig.std(),
                                                                                                                           brier_sig.mean(),brier_sig.std(),
                                                                                                                           ece_sig.mean(),ece_sig.std()) )

In [ ]:
clf_Ada_refit = Model_Ada_refit.fit(X_train, y_train)

y_pred_Ada_refit = clf_Ada_refit.predict(X_test)
proba_Ada_uncalibrated_refit = clf_Ada_refit.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_Ada_refit.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_Ada_refit))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_Ada_refit))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_Ada_refit))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_Ada_refit))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_Ada_uncalibrated_refit[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_Ada_uncalibrated_refit))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_Ada_uncalibrated_refit[:,1]))

In [ ]:
reliability("ADA",y_test, proba_Ada_uncalibrated_refit, 10, 'quantile')

## Isotonic

In [ ]:
# Fit model with best parameters
Model_Ada_cali = AdaBoostClassifier(random_state=42)
Model_Ada_cali.set_params(**best_para_ada)

In [ ]:
# Calibrated with Isotonic Calibration 
clf_Ada_iso = CalibratedClassifierCV(base_estimator=Model_Ada_cali, cv=4, method='isotonic')

clf_Ada_iso.fit(X_train, y_train)

y_pred_Ada_iso = clf_Ada_iso.predict(X_test)
proba_Ada_iso = clf_Ada_iso.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_Ada_iso.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_Ada_iso))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_Ada_iso))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_Ada_iso))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_Ada_iso))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_Ada_iso[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_Ada_iso))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_Ada_iso[:,1]))

In [ ]:
reliability("ADA Calibrated (training)", y_train, clf_Ada_iso.predict_proba(X_train), 10,'quantile')

In [ ]:
reliability("ADA (Isotonic)",y_test, proba_Ada_iso, 10, 'quantile')

## Platt

In [ ]:
# Calibrated with sigmoid calibration
clf_Ada_sigmoid = CalibratedClassifierCV(base_estimator=Model_Ada_cali, cv=4, method='sigmoid')

clf_Ada_sigmoid.fit(X_train, y_train)

y_pred_Ada_sigmoid = clf_Ada_sigmoid.predict(X_test)
proba_Ada_sigmoid = clf_Ada_sigmoid.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_Ada_sigmoid.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_Ada_sigmoid))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_Ada_sigmoid))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_Ada_sigmoid))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_Ada_sigmoid))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_Ada_sigmoid[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_Ada_sigmoid))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_Ada_sigmoid[:,1]))

In [ ]:
reliability("ADA Calibrated Platt (training)",y_train, clf_Ada_sigmoid.predict_proba(X_train), 10,'quantile')

In [ ]:
reliability("ADA (Platt)", y_test, proba_Ada_sigmoid, 10)

# Non-linear SVM

## Normal

###tuning svm

In [ ]:
p_grid = {
            'nu': [0.1, 0.25, 0.5, 1.0, ],
            'kernel': ['linear', 'rbf', 'sigmoid',],
            'gamma': ['auto', 'scale']
         }

cv = GroupKFold(n_splits=4)

Model_NuSVM = NuSVC(random_state=42, probability=True)

clf_NuSVM = RandomizedSearchCV(estimator = Model_NuSVM, param_distributions = p_grid, n_iter = 100, cv = cv, scoring = 'brier_score_loss', verbose=2, random_state=42, n_jobs = -1)

clf_NuSVM.fit(X_train, y_train, groups_X_train)

print(clf_NuSVM.best_score_)
print(clf_NuSVM.best_params_)

In [ ]:
best_para_SVM = {'nu': 0.1, 'kernel': 'linear', 'gamma': 'auto'}
# best_para_SVM = clf_NuSVM.best_params_

In [ ]:
# Fit model with best parameters
Model_NuSVM_refit = NuSVC(random_state=42, probability=True)
Model_NuSVM_refit.set_params(**best_para_SVM)

NuSVC(break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
      max_iter=-1, nu=0.1, probability=True, random_state=42, shrinking=True,
      tol=0.001, verbose=False)

In [ ]:
auc_svm,logloss,brier,ece,prop_train,prop_test,auc_iso,logloss_iso,brier_iso,ece_iso,auc_sig,logloss_sig,brier_sig,ece_sig = iterate_models(Model_NuSVM_refit,100,0.2)

In [ ]:
auc_iso

In [ ]:
auc_sig

In [ ]:
auc_svm

In [ ]:
# %%timeit

for size in [0.2,0.3,0.5,0.8]:
  auc,logloss,brier,ece,prop_train,prop_test,auc_iso,logloss_iso,brier_iso,ece_iso,auc_sig,logloss_sig,brier_sig,ece_sig = iterate_models(Model_NuSVM_refit,10,size)
  print('*'*70)

  print("\tTest Set Size: %.2f\n\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (size,
                                                                                                                           auc.mean(),auc.std(),
                                                                                                                           logloss.mean(),logloss.std(),
                                                                                                                           brier.mean(),brier.std(),
                                                                                                                           ece.mean(),ece.std()) )
  print("\n")
  print("\tAUC(min): %1.3f      \tAUC(max): %1.3f"    % (auc.min(),auc.max()))
  print("\tlog_loss(min): %1.3f  \tlog_loss(max): %1.3f" % (logloss.min(),logloss.max()))
  print("\tBrier(min): %1.3f    \tBrier(max): %1.3f"   % (brier.min(),brier.max()))
  print("\tECE(min): %1.3f       \tECE(max): %1.3f"      % (ece.min(),ece.max()))
  print("\tProp train: %.2f \tProp test: %.2f" % (prop_train.mean(), prop_test.mean()))  
  print("\nIsotonic Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_iso.mean(),auc_iso.std(),
                                                                                                                           logloss_iso.mean(),logloss_iso.std(),
                                                                                                                           brier_iso.mean(),brier_iso.std(),
                                                                                                                           ece_iso.mean(),ece_iso.std()) )
  print("\nSigmoid Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_sig.mean(),auc_sig.std(),
                                                                                                                           logloss_sig.mean(),logloss_sig.std(),
                                                                                                                           brier_sig.mean(),brier_sig.std(),
                                                                                                                           ece_sig.mean(),ece_sig.std()) )

NameError: ignored

In [ ]:
clf_NuSVM_refit = Model_NuSVM_refit.fit(X_train, y_train)

y_pred_NuSVM_refit = clf_NuSVM_refit.predict(X_test)
proba_NuSVM_uncalibrated_refit = clf_NuSVM_refit.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_NuSVM_refit.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_NuSVM_refit))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_NuSVM_refit))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_NuSVM_refit))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_NuSVM_refit))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_NuSVM_uncalibrated_refit[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_NuSVM_uncalibrated_refit))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_NuSVM_uncalibrated_refit[:,1]))

In [ ]:
reliability("NuSVM (tunned)", y_test, proba_NuSVM_uncalibrated_refit, 10,'quantile')


### Isotonic

In [ ]:
# Fit model with best parameters
Model_NuSVM_cali = NuSVC(random_state=42, probability=True)
Model_NuSVM_cali.set_params(**best_para_SVM)

In [ ]:
# Calibrated with Isotonic Calibration 
clf_NuSVM_iso = CalibratedClassifierCV(base_estimator=Model_NuSVM_cali, cv=4, method='isotonic')

clf_NuSVM_iso.fit(X_train, y_train)

y_pred_NuSVM_iso = clf_NuSVM_iso.predict(X_test)
proba_NuSVM_iso = clf_NuSVM_iso.predict_proba(X_test)

In [ ]:
Counter(y_pred_NuSVM_iso)

In [ ]:
print("\tscore: %1.3f" % clf_NuSVM_iso.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_NuSVM_iso))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_NuSVM_iso))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_NuSVM_iso))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_NuSVM_iso))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_NuSVM_iso[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_NuSVM_iso))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_NuSVM_iso[:,1]))

In [ ]:
reliability("NuSVM Uncalibrated", y_test, proba_NuSVM_uncalibrated_refit, 10,'quantile')

In [ ]:
reliability("SVM Calibrated Isotonic (training)",y_train, clf_NuSVM_iso.predict_proba(X_train), 10, 'quantile')

In [ ]:
reliability("NuSVM Calibrated Isotonic", y_test, proba_NuSVM_iso, 10, 'quantile')

### Sigmoid (Platt)

In [ ]:
# Calibrated with sigmoid calibration
clf_SVM_sigmoid = CalibratedClassifierCV(base_estimator=Model_NuSVM_cali, cv=4, method='sigmoid')

clf_SVM_sigmoid.fit(X_train, y_train)

y_pred_SVM_sigmoid = clf_SVM_sigmoid.predict(X_test)
proba_SVM_sigmoid = clf_SVM_sigmoid.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_SVM_sigmoid.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_SVM_sigmoid))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_SVM_sigmoid))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_SVM_sigmoid))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_SVM_sigmoid))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_SVM_sigmoid[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_SVM_sigmoid))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_SVM_sigmoid[:,1]))

In [ ]:
reliability("NuSVM Uncalibrated", y_test, proba_NuSVM_uncalibrated_refit, 10,'quantile')

In [ ]:
reliability("SVM Calibrated Platt (training)", y_train, clf_SVM_sigmoid.predict_proba(X_train), 10,'quantile')

In [ ]:
reliability("NuSVM Calibrated Platt", y_test, proba_SVM_sigmoid, 10,'quantile')

## weighted

###tuning svm

In [ ]:
p_grid = {
            'nu': [0.1, 0.25, 0.5, 1.0, ],
            'kernel': ['linear', 'rbf', 'sigmoid',],
            'gamma': ['auto', 'scale']
         }

cv = GroupKFold(n_splits=4)

Model_NuSVM = NuSVC(random_state=42, probability=True, class_weight='balanced')

clf_NuSVM = RandomizedSearchCV(estimator = Model_NuSVM, param_distributions = p_grid, n_iter = 100, cv = cv, scoring = 'brier_score_loss', verbose=2, random_state=42, n_jobs = -1)

clf_NuSVM.fit(X_train, y_train, groups_X_train)

print(clf_NuSVM.best_score_)
print(clf_NuSVM.best_params_)

In [ ]:
best_para_SVM = {'nu': 0.1, 'kernel': 'linear', 'gamma': 'auto'}


# best_para_SVM = clf_NuSVM.best_params_

In [ ]:
# Fit model with best parameters
Model_NuSVM_refit = NuSVC(random_state=42, probability=True, class_weight='balanced')
Model_NuSVM_refit.set_params(**best_para_SVM)

In [ ]:
auc_svm,logloss,brier,ece,prop_train,prop_test,auc_iso,logloss_iso,brier_iso,ece_iso,auc_sig,logloss_sig,brier_sig,ece_sig = iterate_models(Model_NuSVM_refit,100,0.2)

In [ ]:
auc_svm

In [ ]:
auc_iso

In [ ]:
auc_sig

In [ ]:
for size in [0.2,0.3,0.5,0.8]:
  auc,logloss,brier,ece,prop_train,prop_test,auc_iso,logloss_iso,brier_iso,ece_iso,auc_sig,logloss_sig,brier_sig,ece_sig = iterate_models(Model_NuSVM_refit,10,size)
  print('*'*70)

  print("\tTest Set Size: %.2f\n\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (size,
                                                                                                                           auc.mean(),auc.std(),
                                                                                                                           logloss.mean(),logloss.std(),
                                                                                                                           brier.mean(),brier.std(),
                                                                                                                           ece.mean(),ece.std()) )
  print("\n")
  print("\tAUC(min): %1.3f      \tAUC(max): %1.3f"    % (auc.min(),auc.max()))
  print("\tlog_loss(min): %1.3f  \tlog_loss(max): %1.3f" % (logloss.min(),logloss.max()))
  print("\tBrier(min): %1.3f    \tBrier(max): %1.3f"   % (brier.min(),brier.max()))
  print("\tECE(min): %1.3f       \tECE(max): %1.3f"      % (ece.min(),ece.max()))
  print("\tProp train: %.2f \tProp test: %.2f" % (prop_train.mean(), prop_test.mean()))  
  print("\nIsotonic Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_iso.mean(),auc_iso.std(),
                                                                                                                           logloss_iso.mean(),logloss_iso.std(),
                                                                                                                           brier_iso.mean(),brier_iso.std(),
                                                                                                                           ece_iso.mean(),ece_iso.std()) )
  print("\nSigmoid Calibration")
  print("\tMean AUC: %1.3f(%1.3f) \n\tMean log_loss: %1.3f(%1.3f)\n\tMean Brier: %1.3f(%1.3f) \n\tMean ECE: %1.3f(%1.3f)" % (auc_sig.mean(),auc_sig.std(),
                                                                                                                           logloss_sig.mean(),logloss_sig.std(),
                                                                                                                           brier_sig.mean(),brier_sig.std(),
                                                                                                                           ece_sig.mean(),ece_sig.std()) )

In [ ]:
clf_NuSVM_refit = Model_NuSVM_refit.fit(X_train, y_train)

y_pred_NuSVM_refit = clf_NuSVM_refit.predict(X_test)
proba_NuSVM_uncalibrated_refit = clf_NuSVM_refit.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_NuSVM_refit.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_NuSVM_refit))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_NuSVM_refit))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_NuSVM_refit))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_NuSVM_refit))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_NuSVM_uncalibrated_refit[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_NuSVM_uncalibrated_refit))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_NuSVM_uncalibrated_refit[:,1]))

In [ ]:
reliability("NuSVM (tunned)", y_test, proba_NuSVM_uncalibrated_refit, 10)

### Isotonic

In [ ]:
# Fit model with best parameters
Model_NuSVM_cali = NuSVC(random_state=42, probability=True)
Model_NuSVM_cali.set_params(**best_para_SVM)

In [ ]:
# Calibrated with Isotonic Calibration 
clf_NuSVM_iso = CalibratedClassifierCV(base_estimator=Model_NuSVM_cali, cv=4, method='isotonic')

clf_NuSVM_iso.fit(X_train, y_train)

y_pred_NuSVM_iso = clf_NuSVM_iso.predict(X_test)
proba_NuSVM_iso = clf_NuSVM_iso.predict_proba(X_test)

In [ ]:
Counter(y_pred_NuSVM_iso)

In [ ]:
print("\tscore: %1.3f" % clf_NuSVM_iso.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_NuSVM_iso))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_NuSVM_iso))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_NuSVM_iso))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_NuSVM_iso))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_NuSVM_iso[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_NuSVM_iso))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_NuSVM_iso[:,1]))

In [ ]:
reliability("NuSVM Uncalibrated", y_test, proba_NuSVM_uncalibrated_refit, 10)

In [ ]:
reliability("SVM Calibrated Isotonic (training)", y_train, clf_NuSVM_iso.predict_proba(X_train), 10)

In [ ]:
reliability("NuSVM Calibrated Isotonic", y_test, proba_NuSVM_iso, 10)

### Sigmoid (Platt)

In [ ]:
# Calibrated with sigmoid calibration
clf_SVM_sigmoid = CalibratedClassifierCV(base_estimator=Model_NuSVM_cali, cv=4, method='sigmoid')

clf_SVM_sigmoid.fit(X_train, y_train)

y_pred_SVM_sigmoid = clf_SVM_sigmoid.predict(X_test)
proba_SVM_sigmoid = clf_SVM_sigmoid.predict_proba(X_test)

In [ ]:
print("\tscore: %1.3f" % clf_SVM_sigmoid.score(X_test, y_test))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_SVM_sigmoid))
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_SVM_sigmoid))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_SVM_sigmoid))
print("\tbal_acc: %1.3f" % balanced_accuracy_score(y_test, y_pred_SVM_sigmoid))
print("\tauc: %1.3f" % roc_auc_score(y_test, proba_SVM_sigmoid[:,1]))
print("\tlog_loss: %1.3f" % log_loss(y_test, proba_SVM_sigmoid))
print("\tBrier: %1.3f" % brier_score_loss(y_test, proba_SVM_sigmoid[:,1]))

In [ ]:
reliability("NuSVM Uncalibrated", y_test,  proba_NuSVM_uncalibrated_refit, 10)

In [ ]:
reliability("SVM Calibrated Platt (training)", y_train, clf_SVM_sigmoid.predict_proba(X_train), 10)

In [ ]:
reliability("NuSVM Calibrated Platt",y_test, proba_SVM_sigmoid, 10)

# Results

In [ ]:
def plot_calibration_curve(est, name, fig_index):
    """Plot calibration curve for est w/o and with calibration. """
    # Calibrated with isotonic calibration
    isotonic = CalibratedClassifierCV(est, cv=4, method='isotonic')

    # Calibrated with sigmoid calibration
    sigmoid = CalibratedClassifierCV(est, cv=4, method='sigmoid')

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

    for clf, name in [(est, name),
                      (isotonic, name + ' + Isotonic'),
                      (sigmoid, name + ' + Sigmoid')]:
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        prob_pos = clf.predict_proba(X_test)[:, 1]

        # clf_score = brier_score_loss(y_test, prob_pos, pos_label=y.max())
        clf_score = expected_calibration_error(y_test, prob_pos,13)
        fraction_of_positives, mean_predicted_value = \
            calibration_curve(y_test, prob_pos, n_bins=13, strategy='quantile')

        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % (name, clf_score))

        ax2.hist(prob_pos, range=(0, 1), bins=13, label=name,
                 histtype="step", lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")
    ax2.legend(loc="upper center", ncol=2)

    plt.tight_layout()

In [ ]:
plot_calibration_curve(Model_GB, "Gradient Boosting", 1)
plt.savefig('gb_calibration.png', bbox_inches = 'tight')
files.download("gb_calibration.png") 

In [ ]:
plot_calibration_curve(Model_Ada_refit, "ADA", 1)
plt.savefig('ada_calibration.png', bbox_inches = 'tight')
files.download("ada_calibration.png") 